In [8]:
import requests
from json import JSONDecoder
import datetime
import cv2
import os
import time
import _thread
from PIL import Image,ImageDraw,ImageFont

In [9]:
def getpoints(filepath):
    
    #Upload the picture by filepath then recieve the skeleton points.
    
    files={"image_file":open(filepath,"rb")}
    data={"api_key":KEY,"api_secret":SECRET}
    starttime = datetime.datetime.now()
    http_url="https://api-cn.faceplusplus.com/humanbodypp/v1/skeleton"
    response = requests.post(http_url,data=data,files=files)
    endtime = datetime.datetime.now()
    print((endtime-starttime).seconds)

    req_con=response.content.decode("utf-8")
    req_dict=JSONDecoder().decode(req_con)

    return req_dict


In [10]:
def standard(data):
    #standlise the recevive data 
    json=data["skeletons"][0]["landmark"]
    #req_dict=JSONDecoder().decode(json)
    return json


def drawPoints(points):
    
    #draw the points on the origin picture and show it.
    
    #load the picture
    pil_image=Image.open(filepath).convert("RGB")
    draw=ImageDraw.Draw(pil_image)

    #draw body rectangle
    rect=points["skeletons"][0]['body_rectangle']
    left = rect['left']
    top = rect['top']
    right = rect['width']+left
    bottom = rect['height']+top
    draw.rectangle(((left,top),(right,bottom)),outline=(255,0,0))

    font1=ImageFont.truetype('simsun.ttc',15)
    #font1=ImageFont.truetype('symbol.ttf',15)
    text1="("+str(left)+","+str(top)+")"
    draw.text((left+2,top+5),text1,fill=(255,0,255),font=font1)
    text2="("+str(right)+","+str(bottom)+")"
    draw.text((right-2,bottom-5),text2,fill=(255,0,255),font=font1)
    #draw points
    points2=standard(points)
    
    for key,value in points2.items():
        x=value['x']+left
        y=value['y']+top
        print(key+", ("+str(x)+","+str(y)+")")

        draw.ellipse(((x-2,y-2),(x+2,y+2)),fill=(255,0,255))
        text=key+"("+str(x)+","+str(y)+")"
        draw.text((x+2,y-5),text,fill=(245,245,220),font=font1)


    del draw
    pil_image.show()
    
#test_points=getpoints(filepath)
#print(test_points)
#drawPoints(test_points)

In [11]:
def detect(filepath):
    http_url="https://api-cn.faceplusplus.com/facepp/v3/detect"
    files={"image_file":open(filepath,"rb")}
    data={"api_key":KEY,"api_secret":SECRET,"return_attributes":"gender,headpose,eyegaze,eyestatus"}
    
    starttime = datetime.datetime.now()
    response = requests.post(http_url,data=data,files=files)
    endtime = datetime.datetime.now()
    #print((endtime-starttime).seconds)

    req_con=response.content.decode("utf-8")
    req_dict=JSONDecoder().decode(req_con)
    return req_dict


def getEyeStatus(status):
    eye_status=status['faces'][0]['attributes']['eyestatus']
    left_eye=eye_status['left_eye_status']
    right_eye=eye_status['right_eye_status']

    temp=0
    for key,value in left_eye.items():
        if value>temp:
            temp=value
            left_eye_status=key
    #print(left_eye_status)

    if value>=80 and (left_eye_status=='normal_glass_eye_close' or left_eye_status=='no_glass_eye_close'):
        left_eye_status="close"
    else:
        left_eye_status="open"
    
    temp=0
    for key,value in right_eye.items():
        if value>temp:
            temp=value
            right_eye_status=key
        
    if value>=80 and (right_eye_status=='normal_glass_eye_close' or right_eye_status=='no_glass_eye_close'):
        right_eye_status="close"
    else:
        right_eye_status="open"
    return left_eye_status,right_eye_status


In [12]:
def getHeadStatus(status):
    
    #return the pitch angle of head
    
    head=status['faces'][0]['attributes']['headpose']
    head_pitch_angle=head['pitch_angle']
    return head_pitch_angle   

#status=detect("C:\\Users\\18037\\Desktop\\eye01.jpg")
#getHeadStatus(status)

In [13]:
def getStatus(points):
    
    #return the status of a student
    
    status=0
    #print(points)
    if len(points['skeletons'])!=0:
        rect=points["skeletons"][0]['body_rectangle']
        left = rect['left']
        top = rect['top']
        right = rect['width']+left
        bottom = rect['height']+top
        width=rect['width']
        height=rect['height']

        #参照(头颈距离)
        consult=points["skeletons"][0]['landmark']['neck']['y']-points["skeletons"][0]['landmark']['head']['y']
        #print(consult)

        points2=standard(points)

        left_elbow=points2['left_elbow']
        left_hand=points2['left_hand']
        right_elbow=points2['right_elbow']
        right_hand=points2['right_hand']
        #通过手与肘的位置关系，判断学生是否站立
        if left_hand['y']-left_elbow['y']>0.6*consult or right_hand['y']-right_elbow['y']>0.6*consult:
            status=1
            return status
        
        #通过判断截取的人体线框的长宽关系
        if width>height:
            status=6
            return status
    
    detect_status=detect(filepath)
    print(detect_status)
    if len(detect_status['faces'])!=0:
        #根据眼睛情况判断是否睡觉
        (left_eye_status,right_eye_status)=getEyeStatus(detect_status)
        if left_eye_status=="close" or right_eye_status=="close":
            status=2
        else:
            #是否趴桌子3

            #根据偷得角度判断听讲4or读写5
            head_angle=getHeadStatus(detect_status)
            if head_angle<-5:
                status=4
            elif head_angle>5:
                status=5


    return status


In [14]:
KEY="VPCVp0TurMQ1qehFcqcoH-N4J_ngpZaj"
SECRET="G2BpBymeAWFV13W5d_J0e9jQyXAzG9xa"
filepath="F:\\Jupyter\\face\\behaviour_detect\\pic\\test10.jpg"

dic={0:"未检测到人体",1:"站立",2:"睡觉",3:"趴桌子",4:"听讲",5:"读写",6:"趴桌子"}

if __name__=="__main__":
    
    points=getpoints(filepath) 
    result=getStatus(points)
    print(dic[result])

0
fuck
趴桌子
